In [2]:
import networkx as nx
import numpy as np
import pydot
from graphkernels.kernels import RWKernel
from sklearn.metrics.pairwise import euclidean_distances

def dot_to_networkx(dot_graph):
    return nx.nx_pydot.from_pydot(dot_graph)

def adjacency_matrix(graph):
    return nx.to_numpy_array(graph, dtype=np.float64)

def random_walk_kernel(graph1, graph2, l=1):
    adjacency_matrices = [adjacency_matrix(graph1), adjacency_matrix(graph2)]
    kernel_matrix = RWKernel(adjacency_matrices, l=l)
    return kernel_matrix

def kernel_distance(kernel_matrix):
    return euclidean_distances(kernel_matrix)

# Example usage
dot_graph1 = pydot.graph_from_dot_data('''
digraph graphname {
2 [label="category_names: ['SecurityUpdates', 'CriticalUpdates']\ncategory_names: ['SecurityUpdates', 'CriticalUpdates']\nreboot: True\n"];
1 [label="name: Attempt to install Windows updates\n"];
1 -> 2 [label="win_updates"];
}
''')[0]

dot_graph2 = pydot.graph_from_dot_data('''
digraph graphname {
2 [label="category_names: ['SecurityUpdates', 'CriticalUpdates']\ncategory_names: ['SecurityUpdates', 'CriticalUpdates']\nreboot: True\n"];
1 [label="name: Attempt to install Windows updates\n"];
1 -> 2 [label="win_updates"];
}
''')[0]

ModuleNotFoundError: No module named 'graphkernels'

In [ ]:
graph1 = dot_to_networkx(dot_graph1)
graph2 = dot_to_networkx(dot_graph2)

kernel_matrix = random_walk_kernel(graph1, graph2, l=1)
dist = kernel_distance(kernel_matrix)
print("Kernel distance:", dist[0, 1])

In [18]:
"""
=============================================================
Graph classification on MUTAG using the shortest path kernel.
=============================================================
Script makes use of :class:`grakel.ShortestPath`
"""
from __future__ import print_function
print(__doc__)

import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

from grakel.datasets import fetch_dataset
from grakel.kernels import ShortestPath

# Loads the MUTAG dataset
MUTAG = fetch_dataset('MUTAG', data_home='/data/minh/CompilierFuzzing/similarity/metric/', verbose=False, download_if_missing=False)
G, y = MUTAG.data, MUTAG.target

# Splits the dataset into a training and a test set
G_train, G_test, y_train, y_test = train_test_split(G, y, test_size=0.1, random_state=42)

# Uses the shortest path kernel to generate the kernel matrices
gk = ShortestPath(normalize=True)
K_train = gk.fit_transform(G_train)
K_test = gk.transform(G_test)

# Uses the SVM classifier to perform classification
clf = SVC(kernel="precomputed")
clf.fit(K_train, y_train)
y_pred = clf.predict(K_test)

# Computes and prints the classification accuracy
acc = accuracy_score(y_test, y_pred)
print("Accuracy:", str(round(acc*100, 2)) + "%")


Graph classification on MUTAG using the shortest path kernel.
Script makes use of :class:`grakel.ShortestPath`

Accuracy: 94.74%


In [15]:
import networkx as nx
import pydot
from grakel import GraphKernel, Graph
from grakel.kernels import WeisfeilerLehman

def dot_to_networkx(dot_graph):
    return nx.nx_pydot.from_pydot(dot_graph)

def networkx_to_grakel(graph):
    nodes = sorted(graph.nodes(data=True))
    node_mapping = {old: new for new, old in enumerate(graph.nodes())}
    node_labels = {node_mapping[n[0]]: n[1].get('label', str(n[0])) for n in nodes}
    edge_labels = {(node_mapping[u], node_mapping[v]): graph[u][v].get('label', 'edge') for u, v in graph.edges()}
    numerical_edge_labels = {k: hash(v) for k, v in edge_labels.items()}
    edges = [(node_mapping[u], node_mapping[v], numerical_edge_labels.get((node_mapping[u], node_mapping[v]), 0)) for u, v in graph.edges()]
    return Graph(edges, node_labels)

# Example DOT graphs
dot_graph1_str = '''
digraph graphname {
4 [label="CriticalUpdates"];
5 [label="SecurityUpdates"];
3 [label=""];
3 -> 4 [label="category_names"];
3 -> 5 [label="category_names"];
4 -> 5 [style="dashed"];
2 [label="name: Run Windows Update\n"];
2 -> 3 [label="win_updates"];
1 [label="become: True\nhosts: windows_servers\nname: Example playbook\n"];
1 -> 2 [label="tasks"];
}
'''

dot_graph2_str = '''
digraph graphname {
8 [label="msg: my_var is defined\n"];
7 [label="when: my_var is defined\n"];
7 -> 8 [label="debug"];
10 [label="nested_var: {{ undefined_var }}\n"];
9 [label=""];
9 -> 10 [label="my_var"];
6 [label="gather_facts: False\nhosts: localhost\n"];
6 -> 7 [label="tasks"];
6 -> 9 [label="vars"];
}
'''

dot_graph3_str = '''
digraph graphname {
161 [label="msg: Important value is less than 5. Bug report may not be accurate.\n"];
160 [label="name: Check important value\nwhen: important_value < 5\n"];
160 -> 161 [label="debug"];
163 [label="msg: Bug report generated.\n"];
162 [label="name: Generate bug report\n"];
162 -> 163 [label="debug"];
164 [label="important_value: 10\n"];
159 [label="gather_facts: False\nhosts: localhost\nname: Conditional bug report\n"];
159 -> 160 [label="tasks"];
159 -> 162 [label="tasks"];
160 -> 162 [style="dashed"];
159 -> 164 [label="vars"];
}
'''

In [16]:
dot_graph1 = pydot.graph_from_dot_data(dot_graph1_str)[0]
dot_graph2 = pydot.graph_from_dot_data(dot_graph2_str)[0]
dot_graph3 = pydot.graph_from_dot_data(dot_graph3_str)[0]

In [17]:
# Convert DOT to NetworkX and then to GraKeL
graph1_nx = dot_to_networkx(dot_graph1)
graph2_nx = dot_to_networkx(dot_graph2)
graph3_nx = dot_to_networkx(dot_graph3)

/tmp/ipykernel_377831/2320508653.py:7: DeprecationWarning: nx.nx_pydot.from_pydot depends on the pydot package, which hasknown issues and is not actively maintained.

See https://github.com/networkx/networkx/issues/5723
  return nx.nx_pydot.from_pydot(dot_graph)


In [25]:
from networkx.drawing.nx_pydot import read_dot
import io
graph1_nx = read_dot(io.StringIO(dot_graph1_str))
graph2_nx = read_dot(io.StringIO(dot_graph2_str))
graph3_nx = read_dot(io.StringIO(dot_graph3_str))

In [18]:
graph1_grakel = networkx_to_grakel(graph1_nx)
graph2_grakel = networkx_to_grakel(graph2_nx)
graph3_grakel = networkx_to_grakel(graph3_nx)

In [19]:
# Compute the Weisfeiler-Lehman kernel
from grakel.kernels import WeisfeilerLehman
gk = WeisfeilerLehman(n_iter=5, normalize=True)
K = gk.fit_transform([graph1_grakel, graph2_grakel, graph3_grakel])

print("Kernel matrix:", K)

Kernel matrix: [[1.         0.06451613 0.02952693]
 [0.06451613 1.         0.02952693]
 [0.02952693 0.02952693 1.        ]]


In [26]:
data = {
    'dot_graph': [dot_graph1, dot_graph2_str, dot_graph3_str],
    'graph_nx': [graph1_nx, graph2_nx, graph3_nx],
    'graph_grakel': [graph1_grakel, graph2_grakel, graph3_grakel]
}

In [27]:
import pandas as pd
ndf = pd.DataFrame(data)

In [28]:
ndf

,dot_graph,graph_nx,graph_grakel
0,"digraph graphname {\n4 [label=""CriticalUpdates...","(4, 5, 3, 2, 1, \n)",<grakel.graph.Graph object at 0x7efb9f4ea4a0>
1,"\ndigraph graphname {\n8 [label=""msg: my_var i...","(8, 7, 10, 9, 6, \n)",<grakel.graph.Graph object at 0x7efb9f4ea710>
2,"\ndigraph graphname {\n161 [label=""msg: Import...","(161, 160, 163, 162, 164, 159, \n)",<grakel.graph.Graph object at 0x7efb9f4ea440>


In [14]:
# Compute the Random Walk
from grakel.kernels import RandomWalk
gk = RandomWalk(lamda=0.01, normalize=True)
K = gk.fit_transform([graph1_grakel, graph2_grakel])

import numpy as np
K = np.nan_to_num(K)

print("Kernel matrix:", K)

Kernel matrix: [[-1. -1.]
 [-1. -1.]]


In [47]:
# Compute the ShortestPath
from grakel.kernels import ShortestPath
gk = ShortestPath(normalize=True)
K = gk.fit_transform([graph1_grakel, graph2_grakel])

import numpy as np
K = np.nan_to_num(K)

print("Kernel matrix:", K)

Kernel matrix: [[1. 0.]
 [0. 1.]]
